# **Structured Data Preprocessing**

# **Import library yang diperlukan**

In [175]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **Import Dataset**

## **Sumber Dataset: https://www.kaggle.com/mojtaba142/hotel-booking**

In [176]:
os.environ['KAGGLE_USERNAME'] = "akmalzuhdyprasetya"
os.environ['KAGGLE_KEY'] = "3f1fd1ffa5d36294293dfb098b3b7392"

In [177]:
!kaggle datasets download -d mojtaba142/hotel-booking

hotel-booking.zip: Skipping, found more recently modified local copy (use --force to force download)


In [178]:
!unzip -q /content/hotel-booking.zip

replace hotel_booking.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [179]:
data = pd.read_csv("/content/hotel_booking.csv")
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06,Claudia Johnson,Claudia.J@yahoo.com,403-092-5582,************8647
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07,Wesley Aguilar,WAguilar@xfinity.com,238-763-0612,************4333
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07,Mary Morales,Mary_Morales@hotmail.com,395-518-4100,************1821
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07,Caroline Conley MD,MD_Caroline@comcast.net,531-528-1017,************7860


In [180]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

# **1. Encoding Data**

### **Encoding berarti mengubah informasi dari satu format ke format lain. Encoding diperlukan setiap kali kita memiliki nilai kategoris. Encoding akan menetapkan satu nomor unik untuk entitas tertentu. Sebagian besar waktu nilai kategoris dalam bentuk label ( yaitu ya, tidak, benar, salah) Jadi komputer tidak akan menganggapnya sebagai fitur karena komputer bekerja dengan angka. jadi kita harus menetapkan kuantitas numerik untuk setiap kuantitas dan proses itu disebut 'Encoding'. Ada dua jenis teknik Encoding, Label Encoding dan One Hot Encoding.**


In [181]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

In [182]:
data['hotel'].value_counts()

City Hotel      79330
Resort Hotel    40060
Name: hotel, dtype: int64

## **1.1. Label Encoding**

### **Label Encoding mengacu pada konversi label ke dalam bentuk numerik untuk mengubahnya menjadi bentuk yang dapat dibaca mesin. Algoritma machine learning kemudian dapat memutuskan dengan cara yang lebih baik bagaimana label tersebut harus dioperasikan. Ini adalah langkah pra-pemrosesan yang penting untuk kumpulan data terstruktur dalam supervised learning.**

In [183]:
le = LabelEncoder()
data['hotel'] = le.fit_transform(data['hotel'])

In [184]:
data['hotel'].value_counts()

0    79330
1    40060
Name: hotel, dtype: int64

In [185]:
le.classes_

array(['City Hotel', 'Resort Hotel'], dtype=object)

## **1.2. One Hot Encoding**

### **One Hot Encoding adalah salah satu metode konversi data untuk mempersiapkannya untuk suatu algoritma dan mendapatkan prediksi yang lebih baik. Dengan teknik ini, kita mengonversi setiap nilai kategorikal menjadi kolom kategoris baru dan menetapkan nilai biner 1 atau 0 ke kolom tersebut. Setiap nilai integer direpresentasikan sebagai vektor biner. Semua nilai adalah nol, dan indeks ditandai dengan 1. Contoh sederhana dapat dilihat pada gambar di bawah ini.**

<img src='https://drive.google.com/uc?id=1lY3HGhuR2JpAR1cSpiLttFnvRbTWS-N1'>

In [186]:
data['customer_type'].value_counts()

Transient          89613
Transient-Party    25124
Contract            4076
Group                577
Name: customer_type, dtype: int64

In [187]:
one_hot = OneHotEncoder()
transformed_data = one_hot.fit_transform(data['customer_type'].values.reshape(-1,1)).toarray()

In [188]:
one_hot.categories_

[array(['Contract', 'Group', 'Transient', 'Transient-Party'], dtype=object)]

In [189]:
transformed_data = pd.DataFrame(transformed_data, columns = ['Contract', 'Group', 'Transient', 'Transient-Party'])

In [190]:
transformed_data.head()

,Contract,Group,Transient,Transient-Party
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [191]:
transformed_data.iloc[666]

Contract           0.0
Group              0.0
Transient          1.0
Transient-Party    0.0
Name: 666, dtype: float64

# **2. Normalization & Standardization**

### **Sebelum melakukan tahap ini, kita harus menghilangkan atribut yang tidak bernilai numerik.**

In [192]:
# yang dipertimbangkan hanya kolom dengan data yang bernilai numerik
numeric_columns = [c for c in data.columns if data[c].dtype != np.dtype('O')]

In [193]:
len(numeric_columns) , len(data.columns)

(21, 36)

In [194]:
numeric_columns.remove('company')
numeric_columns.remove('agent')

In [195]:
temp_data = data[numeric_columns]

In [196]:
temp_data.head(21)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,342,2015,27,1,0,0,2,0.0,0,0,0,0,3,0,0.00,0,0
1,1,0,737,2015,27,1,0,0,2,0.0,0,0,0,0,4,0,0.00,0,0
2,1,0,7,2015,27,1,0,1,1,0.0,0,0,0,0,0,0,75.00,0,0
3,1,0,13,2015,27,1,0,1,1,0.0,0,0,0,0,0,0,75.00,0,0
4,1,0,14,2015,27,1,0,2,2,0.0,0,0,0,0,0,0,98.00,0,1
5,1,0,14,2015,27,1,0,2,2,0.0,0,0,0,0,0,0,98.00,0,1
6,1,0,0,2015,27,1,0,2,2,0.0,0,0,0,0,0,0,107.00,0,0
7,1,0,9,2015,27,1,0,2,2,0.0,0,0,0,0,0,0,103.00,0,1
8,1,1,85,2015,27,1,0,3,2,0.0,0,0,0,0,0,0,82.00,0,1
9,1,1,75,2015,27,1,0,3,2,0.0,0,0,0,0,0,0,105.50,0,0


## **2.1. Normalization**

### **Normalisasi digunakan untuk menskalakan data suatu atribut sehingga berada dalam rentang yang lebih kecil, seperti -1.0 hingga 1.0 atau 0.0 hingga 1.0. Proses ini umumnya berguna jika kita ingin melakukan algoritma klasifikasi.**

### **Ketika beberapa atribut ada tetapi atribut memiliki nilai pada skala yang berbeda, ini dapat menyebabkan model data yang buruk saat melakukan operasi penambangan data. Jadi mereka dinormalisasi untuk membawa semua atribut pada skala yang sama. Pada proses ini kita menggunakan kelas `MinMaxScaler`**

In [197]:
from sklearn.preprocessing import MinMaxScaler

In [198]:
import warnings
warnings.filterwarnings('ignore')

In [199]:
normalizer = MinMaxScaler()

In [200]:
temp_data.dropna(axis = 1 , inplace = True)

In [201]:
normalized_data = normalizer.fit_transform(temp_data)

In [202]:
pd.DataFrame(normalized_data , columns = temp_data.columns).head(21)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1.0,0.0,0.464043,0.0,0.5,0.0,0.000000,0.00,0.036364,0.0,0.0,0.0,0.0,0.142857,0.0,0.001180,0.0,0.0
1,1.0,0.0,1.000000,0.0,0.5,0.0,0.000000,0.00,0.036364,0.0,0.0,0.0,0.0,0.190476,0.0,0.001180,0.0,0.0
2,1.0,0.0,0.009498,0.0,0.5,0.0,0.000000,0.02,0.018182,0.0,0.0,0.0,0.0,0.000000,0.0,0.015053,0.0,0.0
3,1.0,0.0,0.017639,0.0,0.5,0.0,0.000000,0.02,0.018182,0.0,0.0,0.0,0.0,0.000000,0.0,0.015053,0.0,0.0
4,1.0,0.0,0.018996,0.0,0.5,0.0,0.000000,0.04,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.019307,0.0,0.2
5,1.0,0.0,0.018996,0.0,0.5,0.0,0.000000,0.04,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.019307,0.0,0.2
6,1.0,0.0,0.000000,0.0,0.5,0.0,0.000000,0.04,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.020972,0.0,0.0
7,1.0,0.0,0.012212,0.0,0.5,0.0,0.000000,0.04,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.020232,0.0,0.2
8,1.0,1.0,0.115332,0.0,0.5,0.0,0.000000,0.06,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.016347,0.0,0.2
9,1.0,1.0,0.101764,0.0,0.5,0.0,0.000000,0.06,0.036364,0.0,0.0,0.0,0.0,0.000000,0.0,0.020694,0.0,0.0


## **2.2. Standardization**

### **Standarisasi kumpulan data adalah persyaratan umum untuk banyak estimator dari machine learning yang diterapkan dalam package scikit-learn, mungkin data akan berperilaku buruk jika fitur individu tidak lebih atau kurang terlihat seperti data terdistribusi normal standar serta Gaussian dengan rata-rata nol dan varians unit.**

### **Dengan standardisasi, kita dapat mengambil atribut dengan distribusi Gaussian, distribusi mean, dan distribusi standar deviasi yang berbeda kemudian mengubahnya menjadi distribusi Gaussian standar dengan nilai mean 0 dan standar deviasinya adalah 1. Untuk ini, kita menggunakan kelas `StandardScaler`.**

In [203]:
from sklearn.preprocessing import StandardScaler

In [204]:
standard_scaler = StandardScaler()

In [205]:
standardized_data = standard_scaler.fit_transform(temp_data)

In [206]:
pd.DataFrame(standardized_data , columns = temp_data.columns).head(21)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1.407224,-0.767040,2.227051,-1.634768,-0.012141,-1.685297,-0.928890,-1.310240,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,4.260101,-0.131924,-2.015038,-0.254873,-0.720694
1,1.407224,-0.767040,5.923385,-1.634768,-0.012141,-1.685297,-0.928890,-1.310240,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,5.793131,-0.131924,-2.015038,-0.254873,-0.720694
2,1.407224,-0.767040,-0.907814,-1.634768,-0.012141,-1.685297,-0.928890,-0.786207,-1.478447,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.530935,-0.254873,-0.720694
3,1.407224,-0.767040,-0.851667,-1.634768,-0.012141,-1.685297,-0.928890,-0.786207,-1.478447,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.530935,-0.254873,-0.720694
4,1.407224,-0.767040,-0.842309,-1.634768,-0.012141,-1.685297,-0.928890,-0.262174,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.075810,-0.254873,0.540666
5,1.407224,-0.767040,-0.842309,-1.634768,-0.012141,-1.685297,-0.928890,-0.262174,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.075810,-0.254873,0.540666
6,1.407224,-0.767040,-0.973319,-1.634768,-0.012141,-1.685297,-0.928890,-0.262174,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,0.102282,-0.254873,-0.720694
7,1.407224,-0.767040,-0.889098,-1.634768,-0.012141,-1.685297,-0.928890,-0.262174,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,0.023130,-0.254873,0.540666
8,1.407224,1.303712,-0.177905,-1.634768,-0.012141,-1.685297,-0.928890,0.261858,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,-0.392419,-0.254873,0.540666
9,1.407224,1.303712,-0.271483,-1.634768,-0.012141,-1.685297,-0.928890,0.261858,0.247897,-0.081579,-0.18156,-0.10318,-0.091555,-0.338990,-0.131924,0.072600,-0.254873,-0.720694


# **3. Menghitung Nilai yang Hilang**

### **Menangani nilai yang hilang adalah tugas penting yang harus dilakukan oleh setiap data scientist. Kita dapat menangani nilai yang hilang dengan dua cara, yaitu menghapus data (seluruh baris) yang memiliki nilai yang hilang atau menambahkan nilai dengan menggunakan beberapa strategi atau menggunakan `Imputer`.**

### **Sebelum itu kita harus melihat pada baris mana terdapat yang hilang. Kita dapat mencarinya cukup dengan menggunakan method `.isnull()`**

In [207]:
data.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [208]:
data['agent'].isnull().sum()

16340

## **3.1. Simple Imputer**

### **Kita dapat menghapus nilai yang hilang ketika rasio jumlah nilai yang hilang dan jumlah total nilai itu rendah. Jadi dalam situasi khusus ini, kita dapat menghapus nilai yang hilang menggunakan method `dropna()` yang telah disiapkan oleh `pandas`. Jika rasionya tinggi maka kita harus menghitung nilainya terlebih dahulu.**

### **Di sini kita menggunakan kelas `SimpleImputer`.**

In [209]:
from sklearn.impute import SimpleImputer

In [210]:
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')

In [211]:
agent_col = imputer.fit_transform(data['agent'].values.reshape(-1,1))

In [212]:
pd.DataFrame(agent_col).isnull().sum()

0    0
dtype: int64

In [213]:
data['agent'].isnull().sum()

16340

# **4. Discretization**

### **Diskritisasi (atau dikenal sebagai kuantisasi atau binning) merupakan proses yang menyediakan cara untuk mempartisi fitur kontinu menjadi nilai diskrit. Kumpulan data tertentu dengan atribut yang bernilai kontinu dapat memperoleh manfaat dari proses diskritisasi ini, karena diskritisasi dapat mengubah kumpulan data atribut kontinu menjadi kumpulan data yang hanya memiliki atribut nominal.**

### **Di sini kita akan mencoba menggunakan tiga cara Diskritisasi yaitu Quantile Discretization Transform, Uniform Dicretization Transform, dan KMeans Discretization Transform, dimana masing-masing methodnya telah disiapkan oleh kelas `KBinsDiscretizer`.**

In [214]:
from sklearn.preprocessing import KBinsDiscretizer

In [215]:
temp_data.head(21)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1,0,342,2015,27,1,0,0,2,0,0,0,0,3,0,0.00,0,0
1,1,0,737,2015,27,1,0,0,2,0,0,0,0,4,0,0.00,0,0
2,1,0,7,2015,27,1,0,1,1,0,0,0,0,0,0,75.00,0,0
3,1,0,13,2015,27,1,0,1,1,0,0,0,0,0,0,75.00,0,0
4,1,0,14,2015,27,1,0,2,2,0,0,0,0,0,0,98.00,0,1
5,1,0,14,2015,27,1,0,2,2,0,0,0,0,0,0,98.00,0,1
6,1,0,0,2015,27,1,0,2,2,0,0,0,0,0,0,107.00,0,0
7,1,0,9,2015,27,1,0,2,2,0,0,0,0,0,0,103.00,0,1
8,1,1,85,2015,27,1,0,3,2,0,0,0,0,0,0,82.00,0,1
9,1,1,75,2015,27,1,0,3,2,0,0,0,0,0,0,105.50,0,0


## **4.1 Quantile Discretization Transform**

### **Dalam cara ini setiap bin memiliki jumlah nilai yang sama dan pembagiannya dilakukan berdasarkan persentil.**

In [216]:
trans = KBinsDiscretizer(n_bins =10 , encode = 'ordinal' , strategy='quantile')
new_data = trans.fit_transform(temp_data)

In [217]:
pd.DataFrame(new_data,columns = temp_data.columns)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,0.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,0.0,0.0,2.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,0.0,0.0,2.0,0.0,4.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,2.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
119386,0.0,0.0,6.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,2.0
119387,0.0,0.0,3.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,2.0
119388,0.0,0.0,6.0,1.0,6.0,9.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


## **4.2 Uniform Discretization Transform**

### **Dalam cara ini setiap bin memiliki lebar yang sama dalam rentang nilai untuk variabel.**

In [218]:
trans = KBinsDiscretizer(n_bins =10 , encode = 'ordinal' , strategy='uniform')
new_data = trans.fit_transform(temp_data)

In [219]:
pd.DataFrame(new_data,columns = temp_data.columns)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,9.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,9.0,0.0,9.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,9.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,0.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119386,0.0,0.0,1.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
119387,0.0,0.0,0.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
119388,0.0,0.0,1.0,9.0,6.0,9.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **4.3 KMeans Discretization Transform**

### **Cara ini pertama menggunakan KMeans untuk mempartisi nilai data input ke dalam cluster. Kemudian, strategi diskritisasi untuk data input dibuat dengan menggunakan informasi nilai maksimum dan minimum dari kumpulan data, pusat cluster yang dihitung dan titik tengah antara masing-masing dua cluster.**

In [220]:
trans = KBinsDiscretizer(n_bins =10 , encode = 'ordinal' , strategy='kmeans')
new_data = trans.fit_transform(temp_data)

In [221]:
pd.DataFrame(new_data,columns = temp_data.columns)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
0,1.0,0.0,6.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,9.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0.0,0.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
119386,0.0,0.0,2.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,3.0
119387,0.0,0.0,1.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0
119388,0.0,0.0,2.0,2.0,6.0,9.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
